# Project 16 - Building a Spam Filter with Naive Bayes

In this guided project, we're going to study the practical side of Naive Bayes algorithm by building a spam filter for SMS messages.

Our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

## Reading in the Data

In [1]:
import pandas as pd

sms_spam = pd.read_csv('sms+spam+collection/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Our dataset has 5572 rows, meaning that there are 5572 different messages. On the `Label` column we have information about the message: if the message is spam, the label is `spam`, if it is not a spam message, then the label is `ham`.

In [2]:
sms_spam['Label'].value_counts(normalize=True)

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

From the frequency table above we can see that 13.4% of the messages are spam.

## Training and Test Set

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- <b>A training set</b> is used to "train" the computer how to classify messages
- <b>A test set</b> is used to test how good the spam filter is with classifying new messages

We're going to keep 80% of our dataset for training, and 20% for testing.

<b>The goal of this project is to create a spam filter that classifies new messages with accuracy greater than 80%. </b>

In [3]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
testing_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(testing_set.shape)

print(training_set.head(2))
print(training_set.tail(2))

(4458, 2)
(1114, 2)
  Label                                            SMS
0   ham                   Yep, by the pretty sculpture
1   ham  Yes, princess. Are you going to make me moan?
     Label                                                SMS
4456   ham  Hello my boytoy ... Geeee I miss you already a...
4457   ham                           Wherre's my boytoy ? :-(


Let's next find out the percentage of spam and ham in both the training and the test set to find out if the percentages are similar to what they were earlier in the full dataset.

In [4]:
print(training_set['Label'].value_counts(normalize=True))
print(testing_set['Label'].value_counts(normalize=True))


Label
ham     0.86541
spam    0.13459
Name: proportion, dtype: float64
Label
ham     0.868043
spam    0.131957
Name: proportion, dtype: float64


The percentages are close to the percentages of full dataset (13.4%).

## Letter Case and Punctuation

The next step is to use the training set to teach the algorithm to classify new messages. To calculate the probabilities needed for filtering, we will first need to clean the data. For example some messages look like this:
|label|SMS|
|-----|---|
|spam|SECRET PRIZE! CLAIM SECRET PRICE NOW!!|
|ham|Coming to my secret party?|
|spam|Winner! Claim secret prize now!|

We want to make the data look like this:

|label|secret|price|claim|now|coming|to|my|party|winner|
|-----|------|-----|-----|---|------|--|--|-----|------|
|spam|2|2|1|1|0|0|0|0|0|
|ham|1|0|0|0|1|1|1|1|0|
|spam|1|1|1|1|0|0|0|0|1|

So instead of having `SMS` column, we want a series of new columns that contain unique words (in lower case), and each row has a count of those words for that message. We will also delete all special marks, like `!!!` and `,`.

In [5]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


## List of Unique Words

Next we are going to create a python list that contains all the unique words from all the messages.

In [6]:
print(training_set.head(5))
# \W means all characters that are not a-z A-Z or 0-9
training_set['SMS'] = training_set['SMS'].str.replace(r'\W', " ", regex=True)
training_set['SMS'] = training_set['SMS'].str.lower()
#delete extra spaces
training_set['SMS'] = training_set['SMS'].str.replace(r'\s+', ' ', regex=True)
print('')
print(training_set.head(5))

  Label                                                SMS
0   ham                       Yep, by the pretty sculpture
1   ham      Yes, princess. Are you going to make me moan?
2   ham                         Welp apparently he retired
3   ham                                            Havent.
4   ham  I forgot 2 ask ü all smth.. There's a card on ...

  Label                                                SMS
0   ham                        yep by the pretty sculpture
1   ham        yes princess are you going to make me moan 
2   ham                         welp apparently he retired
3   ham                                            havent 
4   ham  i forgot 2 ask ü all smth there s a card on da...


In [7]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

We now have a list of each unique word, overall there are 7783 unique words. Next we are going to use that vocabulary to make unique word columns.

## The Final Training Set

In [8]:
#dictionary with all unique words
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [19]:
word_counts = pd.DataFrame(word_counts_per_sms)
print(word_counts.shape)
word_counts.head()

(4458, 7783)


,5p,sport,actual,darlings,wallet,interview,jess,is,go,teasing,...,lick,08002988890,poo,area,21870000,among,unique,clocks,ur,person
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


From the table above we can see that we have created a dataframe with 4458 rows and 7783 columns. Now we just need to add the `Label` and `SMS` columns and add all the word counts for each row.

In [10]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,5p,sport,actual,darlings,wallet,interview,jess,is,...,lick,08002988890,poo,area,21870000,among,unique,clocks,ur,person
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

For Naive Bayes Algorithm we will need to know the probability values of P(spam given unique words) and P(non-spam given unique words). To avoid 0 values, we will use Laplace smoothing and add 1 to all the word counts.

Now we need to find the number of words in all the spam messages, number of words in all the non-spam messages and number of all unique words. We already found the number of unique words above, which was 7783. We can use the same code with filtering to get other number values.

In [11]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary 
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1
print(f"P(Spam): {p_spam}")
print(f"P(Ham): {p_ham}")
print(f"Number of spam words: {n_spam}")
print(f"Number of ham words: {n_ham}")
print(f"Number of unique words: {n_vocabulary}")

P(Spam): 0.13458950201884254
P(Ham): 0.8654104979811574
Number of spam words: 15190
Number of ham words: 57237
Number of unique words: 7783


## Calculating Parameters

Next we need to calculate probabilities for each word, for example how often "secret" is in spam messages, and how often "secret" is in ham messages. This means that we have to calculate 15566 probabilities. Calculating these probabilities beforehand makes the algorithm more efficient when new message comes in, because it doesn't have to calculate them every time.

In [20]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

#some examples
print(parameters_spam['installing'])
print(parameters_spam['edition'])
print(parameters_spam['02073162414'])

4.3529360553693465e-05
4.3529360553693465e-05
0.0001305880816610804


Now we have the dictionaries that have the probabilites for different words for both spam and ham.

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn).
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [13]:
import re

def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'Equal proabilities, have a human classify this!'
#This should be spam        
print(classify('WINNER!! This is the secret code to unlock the money: C3421.'))
#This should be ham
print(classify('Sounds good, Tom, then see u there'))

spam
ham


## Spam Filter's Accuracy

Now that we have a working spam filter, we are going to test how well it works on our test set.

In [14]:
testing_set['predicted'] = testing_set['SMS'].apply(classify)
testing_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


From the table above we can see the `Label` column that still contains labels selected by humans. The `predicted` column is our algorithm deciding whether or not it thinks the message was spam or ham. Let's check how accurate it is by dividing all the correct answers with the amount of messages.

In [15]:
correct = 0
total = 0

for index, row in testing_set.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1
        total += 1
    else:
        total += 1

print(f"Correct answers: {correct}")
print(f"Total answers: {total}")
print(f"Accuracy: {correct/total}")

Correct answers: 1100
Total answers: 1114
Accuracy: 0.9874326750448833


We can see that the accuracy is very good, 98.7%. The goal was to make a filter that is at least 80% accurate, so this works a lot better than expected. Out of 1114 message our spam filter classified 1100 correctly.

## Wrong Predictions

In [16]:
for index, row in testing_set.iterrows():
    if row['Label'] != row['predicted']:
        print(f"{index}: {row['SMS']}")

114: Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net
135: More people are dogging in your area now. Call 09090204448 and join like minded guys. Why not arrange 1 yourself. There's 1 this evening. A£1.50 minAPN LS278BB
152: Unlimited texts. Limited minutes.
159: 26th OF JULY
284: Nokia phone is lovly..
293: A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked 'hw cn u run so fast?' D boy replied "Boost is d secret of my energy" n instantly d girl shouted "our energy" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8
302: No calls..messages..missed calls
319: We have sent JD for Customer Service cum Accounts Executive to ur mail id, For details contact u

Above are all the wrong predictions. In my opinion, they are pretty hard to classify even as a human, so the algorithm worked very well. Most of the messages contain realistic-looking conversations, that are hard to identify as a spam, unless you know that they are promoting some sort of website or service, instead of just catching up.